In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
con = sqlite3.connect("./database.sqlite")

In [3]:
data = pd.read_sql_query("""
Select * from Reviews
where Score != 3""", con)

In [4]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
type(data)

pandas.core.frame.DataFrame

In [6]:
data.sort_values?

In [7]:
sorted_data = data.sort_values(by = ['ProductId'])

In [8]:
sorted_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,4,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,4,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,5,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,5,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [9]:
data.shape, sorted_data.shape

((525814, 10), (525814, 10))

In [10]:
sorted_data.drop_duplicates?

In [11]:
sorted_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [12]:
final_data = sorted_data.drop_duplicates(subset ={'UserId', 'ProfileName','Time','Text'}, keep = 'first')

In [13]:
sorted_data.shape, final_data.shape

((525814, 10), (364173, 10))

In [14]:
final_data = final_data[final_data['HelpfulnessNumerator'] <= final_data['HelpfulnessDenominator']]

In [15]:
final_data.shape

(364171, 10)

In [16]:
import re

In [17]:
import string

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [18]:
re.sub?

In [24]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleaned_sentence = re.sub(cleanr, ' ', sentence)
    return cleaned_sentence

def cleanpunc(word):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'', word)
    cleaned = re.sub(r'[.|,|)|(|\|/|)|{|}|:]',r' ', word)
    return cleaned

In [25]:
SnowballStemmer?

In [26]:
sno = SnowballStemmer('english')

In [27]:
w = 'Pqr'
w.lower()

'pqr'

In [28]:
sno.stem?

In [ ]:
i = 0
final_sentences = []

for sent in final_data['Text'].values:
    sent = cleanhtml(sent)
    sqrl = ''
    for w in sent.split():
        if (cleanpunc(w).lower() not in stopwords.word('english')) and (len(w) >2 ):
            w = sno.stem(w)